## PySpark 설치

In [1]:
%pip install pyspark==3.5.6
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wget, os
import pandas as pd

처리할 데이터 파일 다운로드

In [3]:
save_path = 'data/1800.csv'
download_path = 'https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv'

if not os.path.exists(save_path):
    wget.download(download_path, out=save_path)

In [4]:
!chcp 65001
!dir data

Active code page: 65001
 Volume in drive C has no label.
 Volume Serial Number is BC10-1C56

 Directory of c:\Users\chaeyni\Documents\study\spark\programming-pyspark\chapter2\data

2025-07-27  오후 05:13    <DIR>          .
2025-07-27  오후 05:07    <DIR>          ..
2025-07-27  오후 05:13            62,728 1800.csv
2025-07-27  오후 05:04               997 name_gender.csv
               2 File(s)         63,725 bytes
               2 Dir(s)  619,418,148,864 bytes free


In [5]:
!powershell -Command "Get-Content data/1800.csv | Select-Object -First 5"

ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
EZE00100082,18000101,TMAX,-86,,,E,
EZE00100082,18000101,TMIN,-135,,,E,


### 판다스 데이터프레임으로 처리해보기

In [6]:
file_1800_csv = save_path

pd_df = pd.read_csv(
    file_1800_csv,
    names=["stationID", "date", "measure_type", "temperature"],
    usecols=[0, 1, 2, 3]
)

In [7]:
pd_df.head()

,stationID,date,measure_type,temperature
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135


In [8]:
# Filter out all but TMI entries
pd_minTemps = pd_df[pd_df['measure_type'] == "TMIN"]

In [9]:
pd_minTemps.head()

,stationID,date,measure_type,temperature
1,ITE00100554,18000101,TMIN,-148
4,EZE00100082,18000101,TMIN,-135
6,ITE00100554,18000102,TMIN,-125
9,EZE00100082,18000102,TMIN,-130
11,ITE00100554,18000103,TMIN,-46


In [10]:
# Select only stationID and temperature
pd_stationTemps = pd_minTemps[["stationID", "temperature"]]

In [11]:
# Aggregate to find minimum temperature for every station
pd_minTempsByStation = pd_stationTemps.groupby(["stationID"]).min("temperature")
pd_minTempsByStation.head()

,temperature
stationID,
EZE00100082,-135
ITE00100554,-148


## Spark으로 처리해보기

In [12]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [13]:
# 1) 로컬 호스트 IP 구하기
host_ip = "127.0.0.1"  # 또는 socket.gethostbyname(socket.gethostname())

# 2) 환경변수 지정
os.environ['SPARK_LOCAL_IP']           = host_ip
os.environ['PYSPARK_PYTHON']           = os.sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']    = os.sys.executable

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #1")
conf.set("spark.master", "local[*]")
conf.set("spark.driver.host", host_ip)
conf.set("spark.driver.bindAddress", host_ip)

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [41]:
df = spark.read.format("csv").load(file_1800_csv) # spark.read.csv(file_1800_csv)

In [42]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [45]:
df = spark.read.format("csv")\
        .load(file_1800_csv)\
        .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [46]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [14]:
df = spark.read.format("csv")\
        .option("inferSchema", "true")\
        .load(file_1800_csv)\
        .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [15]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [17]:
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.types import StructType, StructField

schema = StructType([ \
                    StructField("stationID", StringType(), True), \
                    StructField("date", IntegerType(), True), \
                    StructField("measure_type", StringType(), True), \
                    StructField("temperature", FloatType(), True)])

In [18]:
# df = spark.read.schema(schema).format("csv").load(file_1800_csv)
df = spark.read.schema(schema).csv(file_1800_csv)

In [19]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [20]:
# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

In [21]:
minTemps.count()

730

In [22]:
# Column expression으로 필터링 적용
minTemps = df.where(df.measure_type == "TMIN")

In [23]:
minTemps.count()

730

In [24]:
# SQL expression으로 필터링 적용
minTemps = df.where("measure_type = 'TMIN'")

In [25]:
minTemps.count()

730

In [28]:
# Aggregate to find minimum temperature for every station
minTempsByStation = minTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [30]:
# Select only stationID and temperature
stationTemps = minTemps[["stationID", "temperature"]]

In [33]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [34]:
stationTemps = minTemps.select("stationID", "temperature")

In [35]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [36]:
# Collect, format, and print the results
results = minTempsByStation.collect()

In [37]:
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	-148.00F
EZE00100082	-135.00F


## Spark SQL로 처리해보기

In [38]:
df.createOrReplaceTempView("station1800")

In [40]:
results = spark.sql("""SELECT stationID, MIN(temperature)
                    FROM station1800
                    WHERE measure_type = 'TMIN'
                    GROUP BY 1""").collect()

In [41]:
# pyspark.sql.Row는 DataFrame의 레코드에 해당하며 필드별로 이름이 존재
for r in results:
    print(r)

Row(stationID='ITE00100554', min(temperature)=-148.0)
Row(stationID='EZE00100082', min(temperature)=-135.0)
